In [1]:
#import the entire training data 
#consisting of faces of images, faces extracted out of video frames, faces out of group photos; and augmented images
#entire training data stored as a list with ID; in the format [[ID; image path]] in list girlb
import os, sys
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from skimage import data
from skimage import transform as tf
from skimage.feature import (match_descriptors, corner_harris,
                             corner_peaks, ORB, plot_matches)
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

file_path = (r'C:\Users\Prerna Prakash Gupta\Desktop\faces1\jpeg1')
girla=[]
for root, dirs, files in os.walk(file_path):    
    for name in files: 
        v=os.path.join(root, name)
        girla.append([root[-2:], v])
#print(len(s))
girla

#No. of ID categories=48
r=[]
for i in range(len(girlb)):
    r.append(girla[i][0])
r=pd.Series(r)  

#ENTIRE TRAINING-DATA IN LIST; with ID and image path as example whon below

girlb[1]

['78', 'C:\\Users\\Prerna Prakash Gupta\\Desktop\\faces1\\MODT\\10278.JPG']

In [2]:
#from tarining data list extract hog-features for every image
#save corresponding label of every image in the list

#create empty list to save hog-images
hog_images = []
#create empty list to save hog-features for every image
hog_features = []
#create list to save coresponding label ID for every image
labels=[]

#loop over training data to extract hig feature for every image
for i in girlb:
    #read and extract every image as a numpy array f
    img=cv2.imread(i[1])
    #re-sixe every image to the following size
    resized_img = resize(img, (128,64)) 
    #fd=hig features, hig_image=hog-image
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2, 2), visualize=True, multichannel=True)
    #save all hog images in a list
    hog_images.append(hog_image)
    #save corresponding label ID of every image in same order in the list labels
    labels.append(i[0])
    # save hog features corresponsding to every image and append in the list
    hog_features.append(fd)
#labels is list of ID-names
labels    
#this is list of image features for every image
hog_features


[array([0.01765226, 0.0260488 , 0.04262134, ..., 0.01093239, 0.02595933,
        0.06136712]),
 array([0.38804661, 0.38804661, 0.        , ..., 0.        , 0.        ,
        0.33329709]),
 array([0.16790037, 0.32866078, 0.23558604, ..., 0.02380245, 0.18261008,
        0.19771801]),
 array([0.35868628, 0.        , 0.        , ..., 0.00762423, 0.00737205,
        0.03395671]),
 array([0.43738716, 0.43738716, 0.05351909, ..., 0.        , 0.        ,
        0.08005193]),
 array([0.21803018, 0.00104384, 0.00594162, ..., 0.        , 0.02738905,
        0.07123616]),
 array([0.28768954, 0.28768954, 0.19184147, ..., 0.0373906 , 0.1366576 ,
        0.29857782]),
 array([0.29943869, 0.        , 0.00657941, ..., 0.04598291, 0.03875232,
        0.10065565]),
 array([0.3062108 , 0.        , 0.0723676 , ..., 0.02484691, 0.11544258,
        0.29143187]),
 array([0.06470198, 0.02754345, 0.02589576, ..., 0.00513584, 0.04138117,
        0.05148835]),
 array([4.16076209e-01, 0.00000000e+00, 4.02313000

In [4]:
#import relevant libraries
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split

#the independent variable x is hog features and dependent variable y is ID name 
# we scale both x and y and stack them as numpy arrays in a form readable by ski-kit learn models



unscaled_x = np.vstack(hog_features).astype(np.float64)
scaler = StandardScaler().fit(unscaled_x)
x = scaler.transform(unscaled_x)
y = np.hstack(labels)

#create a stratified tarining and test split of the data, with 30% test data to evaluate the model
  
X_train, X_test, y_train, y_test = train_test_split( 
                        x, np.ravel(y), stratify=y,
                test_size = 0.30, random_state = 101)

In [5]:
print ("number of training examples =", X_train.shape[0])
print ("number of test examples =", X_test.shape[0])
print ("X_train shape:", X_train.shape)
print ("X_test shape:", X_test.shape)

number of training examples = 3231
number of test examples = 1385
X_train shape: (3231, 3780)
X_test shape: (1385, 3780)


In [6]:
# Find the unique numbers from the train labels
classes = np.unique(y_train)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)


Total number of outputs :  48
Output classes :  ['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28'
 '29' '30' '31' '32' '33' '34' '36' '38' '40' '42' '44' '46' '48' '50'
 '52' '54' '56' '58' '60' '78']


In [6]:
#Perform hyper-parameter grid search to obtain the most optimal SVM model
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import random
from sklearn.model_selection import learning_curve, GridSearchCV


#Perform hyper-parameter grid search to obtain the most optimal SVM model
parameter_candidates = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]
# Create a classifier object with the classifier and parameter candidates
clf = GridSearchCV(estimator=svm.SVC(), param_grid=parameter_candidates, n_jobs=-1)

# Train the classifier on data1's feature and target data
clf.fit(X_train, y_train)   
print('Best score for data1:', clf.best_score_) 
print('Best C:',clf.best_estimator_.C) 
print('Best Kernel:',clf.best_estimator_.kernel)
print('Best Gamma:',clf.best_estimator_.gamma)

print(clf.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(clf.best_estimator_) 

Best score for data1: 0.962239629439997
Best C: 1
Best Kernel: linear
Best Gamma: scale
{'C': 1, 'kernel': 'linear'}
SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [10]:
#Post hyper-parameter optimization; we run the most optimum SVM model post grid search

#load libraries
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import random
from sklearn.model_selection import learning_curve, GridSearchCV

#Post hyper-parameter optimization; we run the most optimum SVM model post grid search
clf=svm.SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
#fit the model on training data
clf.fit(X_train, y_train)

#evaluate the trained model on test-data
predictions = clf.predict(X_test) 
  
# print classification report 
print(classification_report(y_test, predictions)) 

              precision    recall  f1-score   support

          01       0.94      0.97      0.96        33
          02       1.00      1.00      1.00        29
          03       1.00      0.97      0.98        33
          04       0.90      0.98      0.93        44
          05       1.00      0.75      0.86         4
          06       0.95      1.00      0.97        39
          07       1.00      1.00      1.00         5
          08       1.00      0.94      0.97        47
          09       1.00      1.00      1.00        28
          10       0.98      0.98      0.98        48
          11       1.00      1.00      1.00         9
          12       1.00      1.00      1.00        47
          13       1.00      0.95      0.98        43
          14       1.00      0.96      0.98        45
          15       1.00      1.00      1.00         3
          16       1.00      0.87      0.93        15
          17       1.00      1.00      1.00        36
          18       0.97    

In [14]:
#create a folder for saving models from JOBLIB
import joblib
os.chdir(r'C:\Users\Prerna Prakash Gupta\Desktop\savedmodel')
filename = 'supportVMHOGmodel.sav'
joblib.dump(clf, filename)
 

['supportVMHOGmodel.sav']

In [15]:
# some time later...
# load the model from disk
loaded_model = joblib.load('supportVMHOGmodel.sav')
result = loaded_model.score(X_test, y_test)
#we see 97.1% accuracy on test data
print(result)

0.9711191335740073


In [7]:
#KNN approach on HOG data
#perform hyper-parameter-tuning and extract the most optimal model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import learning_curve, GridSearchCV
leaf_size = [1, 15, 20, 30]
n_neighbors = [3, 5, 7, 10]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=3)
#Fit the model
best_model = clf.fit(X_train,y_train)
#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

grid_predictions = clf.predict(X_test) 
  
# print classification report 
print(classification_report(y_test, grid_predictions))


Best leaf_size: 1
Best n_neighbors: 3
              precision    recall  f1-score   support

          01       0.79      0.91      0.85        33
          02       1.00      1.00      1.00        29
          03       0.82      0.94      0.87        33
          04       0.85      0.91      0.88        44
          05       1.00      0.75      0.86         4
          06       0.89      1.00      0.94        39
          07       0.80      0.80      0.80         5
          08       1.00      0.94      0.97        47
          09       0.96      0.93      0.95        28
          10       0.96      0.96      0.96        48
          11       1.00      0.89      0.94         9
          12       1.00      0.98      0.99        47
          13       1.00      0.93      0.96        43
          14       0.96      0.96      0.96        45
          15       0.67      0.67      0.67         3
          16       1.00      0.80      0.89        15
          17       0.97      1.00      0.99

C:\Users\Prerna Prakash Gupta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#on the most optimal model, train the KNN classifier on training data
knn_2 = KNeighborsClassifier(leaf_size=1, n_neighbors=3)
knnmodel = knn_2.fit(X_train,y_train)

#test the model on training data
y_pred= knnmodel.predict(X_test) 

In [59]:
#create a folder for saving models from JOBLIB
import joblib
os.chdir(r'C:\Users\Prerna Prakash Gupta\Desktop\savedmodel')
filename = 'KNNHOGmodel.sav'
joblib.dump(knnmodel, filename)
# some time later...
# load the model from disk
loaded_model = joblib.load('KNNHOGmodel.sav')
result = loaded_model.score(X_test, y_test)
#we see 92% accuracy on test data

 

['KNNHOGmodel.sav']

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import learning_curve, GridSearchCV
######perform hyper parameter optimization for RF and training training data on best performing model
rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True) 

param_grid = { 'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)

grid_predictions = CV_rfc.predict(X_test) 



C:\Users\Prerna Prakash Gupta\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


{'bootstrap': True, 'max_depth': 100, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 1000}


In [7]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier( max_depth=100, min_samples_leaf=3, min_samples_split=8, n_estimators=1000,  oob_score = True)
rfc=rfc.fit(X_train, y_train)
predict=rfc.predict(X_test) 
print(classification_report(y_test, predict))


              precision    recall  f1-score   support

          01       0.94      0.91      0.92        33
          02       0.94      1.00      0.97        29
          03       0.86      0.91      0.88        33
          04       0.62      0.91      0.74        44
          05       0.00      0.00      0.00         4
          06       0.77      0.95      0.85        39
          07       1.00      0.40      0.57         5
          08       0.95      0.89      0.92        47
          09       0.92      0.86      0.89        28
          10       0.85      0.96      0.90        48
          11       1.00      0.67      0.80         9
          12       0.87      0.98      0.92        47
          13       1.00      0.95      0.98        43
          14       0.89      0.87      0.88        45
          15       0.00      0.00      0.00         3
          16       1.00      0.80      0.89        15
          17       0.84      1.00      0.91        36
          18       0.67    

C:\Users\Prerna Prakash Gupta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
#create a folder for saving models from JOBLIB
import joblib
os.chdir(r'C:\Users\Prerna Prakash Gupta\Desktop\savedmodel')
filename = 'RFCHOGmodel1.sav'
joblib.dump(rfc, filename)
# some time later...
# load the model from disk
loaded_model = joblib.load('RFCHOGmodel1.sav')
result = loaded_model.score(X_test, y_test)
#we see 89% accuracy on test data

['RFCHOGmodel.sav']

In [67]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
#create an instance and fit the model 
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)
Predictions = logmodel.predict(X_test)


print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

          01       0.94      0.97      0.96        33
          02       1.00      1.00      1.00        29
          03       1.00      0.97      0.98        33
          04       0.90      0.98      0.93        44
          05       1.00      0.75      0.86         4
          06       0.95      1.00      0.97        39
          07       1.00      1.00      1.00         5
          08       1.00      0.94      0.97        47
          09       1.00      1.00      1.00        28
          10       0.98      0.98      0.98        48
          11       1.00      1.00      1.00         9
          12       1.00      1.00      1.00        47
          13       1.00      0.95      0.98        43
          14       1.00      0.96      0.98        45
          15       1.00      1.00      1.00         3
          16       1.00      0.87      0.93        15
          17       1.00      1.00      1.00        36
          18       0.97    

C:\Users\Prerna Prakash Gupta\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [68]:
#create a folder for saving models from JOBLIB
import joblib
os.chdir(r'C:\Users\Prerna Prakash Gupta\Desktop\savedmodel')
filename = 'LOGITHOGmodel.sav'
joblib.dump(logmodel, filename)
# some time later...
# load the model from disk
loaded_model = joblib.load('LOGITHOGmodel.sav')
result = loaded_model.score(X_test, y_test)


['LOGITHOGmodel.sav']